In [ ]:
import sqlite3, getpass
import pandas as pd
import polars as pl
from itables import show
# import itables.options as opt
# opt.maxBytes = 131072

In [ ]:
sqlite_dir = getpass.getpass(prompt='Base directory for SQLite files : ')

Base directory for SQLite files :  ········


In [ ]:
#connection = sqlite3.connect(f"{sqlite_dir}synop_b.2024031112.sqlite")

connection = sqlite3.connect(f"{sqlite_dir}2024031318__sqlite")
cursor = connection.cursor()

query = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(query)
tables = cursor.fetchall() # ('DATA',) ('HEADER',) ('OmF',)

for table in tables:
    print(table)

connection.close()

('DATA',)
('HEADER',)
('OmF',)


In [ ]:
# connection = sqlite3.connect(f"{sqlite_dir}synop_b.2024031112.sqlite")
connection = sqlite3.connect(f"{sqlite_dir}2024031318__sqlite")
# burp2rdb

data_table = 'DATA'
header_table = 'HEADER'
# Define the SQL query you want to execute
query_data = f"SELECT * FROM {data_table};"
query_header = f"SELECT * FROM {header_table};"

# Execute the query and store the result in a DataFrame
result_data = pd.read_sql_query(query_data, connection)
result_data_arrow = pd.read_sql_query(query_data, connection, dtype_backend='pyarrow')
result_header = pd.read_sql_query(query_header, connection)

connection.close()

In [ ]:
result_data_pl_df = pl.from_pandas(result_data)

In [ ]:
print(type(result_data))
print(type(result_data_pl_df))

<class 'pandas.core.frame.DataFrame'>
<class 'polars.dataframe.frame.DataFrame'>


In [ ]:
delta_lake_df = pl.scan_delta("tar_swob_no_optimization")

In [ ]:
print(result_data.dtypes)

ID_DATA          int64
ID_OBS           int64
BURP_BTYP        int64
VCOORD         float64
VARNO            int64
VCOORD_TYPE     object
OBSVALUE       float64
FLAG             int64
OMP             object
OMA             object
OBS_ERROR       object
FG_ERROR        object
FSO             object
dtype: object


In [ ]:
print(result_data_pl_df.dtypes)

[Int64, Int64, Int64, Float64, Int64, String, Float64, Int64, String, String, String, String, String]


In [ ]:
# result_data_arrow = pd.read_sql_query(query_data, connection, dtype_backend='pyarrow')
print(result_data_arrow.dtypes)

In [ ]:
# result_data_arrow
# show(result_data_arrow, buttons=["copyHtml5", "csvHtml5", "excelHtml5"]) # ... (downsampled from 453,375x13 ...
show(result_data_pl_df.filter(pl.col("VARNO") == 12004)) # ... (downsampled from 25,065x13 to 856x13 as maxBytes=131072)

In [ ]:
result_data_pl_df

ID_DATA,ID_OBS,BURP_BTYP,VCOORD,VARNO,VCOORD_TYPE,OBSVALUE,FLAG,OMP,OMA,OBS_ERROR,FG_ERROR,FSO
i64,i64,i64,f64,i64,str,f64,i64,str,str,str,str,str
1,1,0,null,10051,null,101600.0,0,null,null,null,null,null
2,1,0,null,11011,null,50.0,0,null,null,null,null,null
3,1,0,null,11012,null,1.0,0,null,null,null,null,null
4,1,0,null,12004,null,288.200012,0,null,null,null,null,null
5,1,0,null,10063,null,4.0,0,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…
453371,26123,0,null,20003,null,509.0,4,null,null,null,null,null
453372,26123,0,null,1013,null,0.0,0,null,null,null,null,null
453373,26123,0,null,10004,null,101020.0,0,null,null,null,null,null


In [ ]:
# result_data_pl_df['VARNO'].unique()
varno_list = result_data_pl_df['VARNO'].unique().to_list()

# Generator expression to pad codes and join them with '|'
varno_list_grep = '|'.join(f"{code:06}" for code in varno_list)

#varno_list_grep = '|'.join(map(str, varno_df))

In [ ]:
varno_list_grep
# to supply to `grep -E` '^(001012|001013|002004|002038|005001|006001|007004|007030|007031|010004|010009|010051|010061|010062|010063|010194|011011|011012|011041|012002|012004|012006|012016|012017|013003|013013|013019|013020|013021|013022|013023|013033|013200|013204|020001|020003|020004|020005|020011|020012|020013|020062|020198|020199|020200|020201|020202|020213|022003|022011|022012|022013|022021|022022|022023|022042|022049)'

'001012|001013|002004|002038|005001|006001|007004|007030|007031|010004|010009|010051|010061|010062|010063|010194|011011|011012|011041|012002|012004|012006|012016|012017|013003|013013|013019|013020|013021|013022|013023|013033|013200|013204|020001|020003|020004|020005|020011|020012|020013|020062|020198|020199|020200|020201|020202|020213|022003|022011|022012|022013|022021|022022|022023|022042|022049'

In [ ]:
code_list_df = pl.read_csv("codes_burp.csv", truncate_ragged_lines=True)

In [ ]:
code_list_df

code,name,units,remainder
i64,str,str,str
1012,"""DIR. MOUVEMENT…","""DEG VRAI""","""0 0 …"
1013,"""VITESSE""",""" STATION OBSER…","""M/S"""
2004,"""TYPE D'INSTR. …","""CODE TABLE""","""0 0 …"
2038,"""METH. MESURE T…","""CODE TABLE""","""0 0 …"
5001,"""LATITUDE (HAUT…","""DEGRE""","""5 -9000000 …"
…,…,…,…
20199,"""TYPE DE NUAGES…","""CODE TABLE""","""0 0 …"
20200,"""HAUTEUR P.R. S…",""" BASE DES NUAG…","""M -"""
20201,"""TYPE DE NUAGES…","""CODE TABLE""","""0 0 …"


In [ ]:
result_ID_OBS = pd.merge(result_data, result_header, on="ID_OBS")


In [ ]:
result_ID_OBS

,ID_DATA,ID_OBS,BURP_BTYP,VCOORD,VARNO,VCOORD_TYPE,OBSVALUE,FLAG,OMP,OMA,...,LAT,LON,CODTYP,DATE,TIME,STATUS,DELAY,RUNN,OARS,ELEV
0,1,1,0,NaN,10051,None,101270.000000,0,None,None,...,54.5,10.6,13,20240304,150000,12583936,0,0,0,8.0
1,2,1,0,NaN,11011,None,90.000000,0,None,None,...,54.5,10.6,13,20240304,150000,12583936,0,0,0,8.0
2,3,1,0,NaN,11012,None,11.000000,0,None,None,...,54.5,10.6,13,20240304,150000,12583936,0,0,0,8.0
3,4,1,0,NaN,12004,None,278.399994,0,None,None,...,54.5,10.6,13,20240304,150000,12583936,0,0,0,8.0
4,5,1,0,NaN,12006,None,277.100006,0,None,None,...,54.5,10.6,13,20240304,150000,12583936,0,0,0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518975,518976,30618,0,NaN,12002,None,298.200012,0,None,None,...,-22.1,153.0,13,20240304,200000,12583936,1431,0,0,0.0
518976,518977,30618,0,NaN,22049,None,304.149994,0,None,None,...,-22.1,153.0,13,20240304,200000,12583936,1431,0,0,0.0
518977,518978,30618,8192,1.0,22003,None,130.000000,0,None,None,...,-22.1,153.0,13,20240304,200000,12583936,1431,0,0,0.0
518978,518979,30618,8192,1.0,22013,None,4.000000,0,None,None,...,-22.1,153.0,13,20240304,200000,12583936,1431,0,0,0.0


In [ ]:
unique_time_values = result_ID_OBS["TIME"].unique()
print(unique_time_values)